# Memento: undo and redo

> Storing every single state of the system

One of the interesting things we can do with memento is implementing **undo/redo functionality**, but we need to store every single state of the system as a memento. For some use cases this is not economical because it would take up too much memory, but for others, we can do this and have the entire snapshot of the system.

We need a mechanism for storing every memento. In this lesson we will use a list. We will expand `BankAccount` with this list and we will also store an index that will map the current state of `BankAccount` to a memento in the list, so that we can undo and redo to any state.

We will also have to modify the `BankAccount`'s methods: `deposit` won't just return a `Memento`, but it will also append it to the state list and update the index.

In [1]:
class Memento:
    """No changes"""
    def __init__(self, balance):
        self.balance = balance


class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.changes = [Memento(self.balance)] # state list
        self.current = 0 # state list index, 0 = initial state

    def deposit(self, amount):
        self.balance += amount
        m = Memento(self.balance)
        self.changes.append(m) # append to state list
        self.current += 1 # increase the index
        return m

    def restore(self, memento):
        if memento: # guard condition
            self.balance = memento.balance
            self.changes.append(memento) # append to state list
            self.current = len(self.changes)-1 # set index to last position of state list

    def __str__(self):
        return f'Balance = {self.balance}'

Note that the `BankAccount` initialization now creates a `Memento` instance which is automatically added as the first element of the state list.

The `restore` utility method is a little trickier: we will now assume that `Memento` can be set to `None` (we'll see why soon) and add a guard condition that will only restore the balance if the passed `memento` is an actual memento, then append the memento to the state list just like `deposit` and then set the index to the last position of the list.

We will now add the `undo` and `redo` methods to `BankAccount`: 

* For `undo`, we need to check that `BankAccount` is in any state other than its original state, then select the previous state/memento from the state list and update the balance with the memento's.
* `redo` is similar, except that we need to check that `BankAccount` is in any state other than the latest state, then select the next state/memento from the state list and update the balance.

Should either `undo` or `redo` faile because their checks fail, they will return `None` (which is why we now assume that `restore` can receive `None` as a memento).

In [2]:
class BankAccount:
    def __init__(self, balance=0):
        self.balance = balance
        self.changes = [Memento(self.balance)]
        self.current = 0

    def deposit(self, amount):
        self.balance += amount
        m = Memento(self.balance)
        self.changes.append(m)
        self.current += 1
        return m

    def restore(self, memento):
        if memento:
            self.balance = memento.balance
            self.changes.append(memento)
            self.current = len(self.changes)-1

    def __str__(self):
        return f'Balance = {self.balance}'

    def undo(self):
        if self.current > 0:
            self.current -= 1 # update index
            m = self.changes[self.current]
            self.balance = m.balance
            return m
        return None

    def redo(self):
        if self.current + 1 < len(self.changes):
            self.current += 1
            m = self.changes[self.current]
            self.balance = m.balance
            return m
        return None

Let's see our new methods in action:

In [3]:
ba = BankAccount(100)
ba.deposit(50)
ba.deposit(25)
print(ba)

ba.undo()
print(f'Undo 1: {ba}')
ba.undo()
print(f'Undo 2: {ba}')
ba.redo()
print(f'Redo 1: {ba}')

Balance = 175
Undo 1: Balance = 150
Undo 2: Balance = 100
Redo 1: Balance = 150
